In [1]:
from pynq import DefaultHierarchy
from pynq import allocate
import numpy as np

class Audio_Block_Driver(DefaultHierarchy):
    def __init__(self,description):
        super().__init__(description)
        self.processed_audio_buffer = 0
        
    def configure (self, fft_size):
        """
            Inicializa el buffer de audio de output

            Parámetros:
            - fft_size (int): El tamaño del FFT en puntos, que debe ser una potencia de 2. Por ejemplo, 1024.
        """
        # Actualizar variables clase
        self.processed_audio_buffer = allocate(fft_size,np.int32) # Crear buffer (64b) para mandar datos al DMA, ahorrar tiempo despues
    
    # Function to receive processed audio data
    def apply_effects(self, input_buffer):
        """
            Aplica efectos al buffer de entrada de audio y devuelve el audio procesado.

            Parámetros:
            - input_buffer: Buffer de entrada de audio que será procesado.

            Retorna:
            - processed_audio_buffer: Buffer de salida con el audio procesado.
        """
        self.audio_dma.sendchannel.transfer(input_buffer)
        self.audio_dma.recvchannel.transfer(self.processed_audio_buffer)
        self.audio_dma.sendchannel.wait()
        self.audio_dma.recvchannel.wait()

        return self.processed_audio_buffer
    
    def gain_set_gain(self, gain):
        """
        Configura el efecto de ganancia en la FPGA.

        Parámetros:
        - gain (float): El factor de ganancia por el cual se multiplicará la señal.
        """
        buffer = allocate(shape=(1,), dtype=np.int32)
        buffer[0] = gain

        # Transferir el buffer usando DMA
        self.gain_dma.sendchannel.transfer(buffer)
        self.gain_dma.sendchannel.wait()
        del buffer
        
    def rect_set_enable(self, enable):
        """
            Configura el encendido/apagado de la rectificación de señal.

            Parámetros:
            - enable (int): 1 para habilitar la rectificación, 0 para deshabilitarla.
        """
        
        buffer = allocate(shape=(1,), dtype=np.int32)
        buffer[0] = enable
        
        # Transferir el buffer usando DMA
        self.rect_dma.sendchannel.transfer(buffer)
        self.rect_dma.sendchannel.wait()
        del buffer
        
    def clip_set_threshold(self, threshold):
        """
            Configura el umbral para el efecto Clipper.

            Parámetros:
            - threshold (float): Valor entre 0 y 1 que se establecerá como 
            el umbral de la señal. El valor 0 desactiva el efecto.
        """
        
        buffer = allocate(shape=(1,), dtype=np.int32)
        
        # Asegurarse de que el valor del umbral esté en el rango correcto
        if threshold > 1:
            threshold = 1
            
        if threshold <= 0:
            threshold = 1
        
        # Convertir el valor del umbral a un entero correspondiente al rango del hardwar
        buffer[0] = int(threshold * 2147483647)
        
        # Transferir el buffer usando DMA
        self.clip_dma.sendchannel.transfer(buffer)
        self.clip_dma.sendchannel.wait()
        del buffer
    
    # Reemplazar el DefaultHierarchy para utilizar la clase de driver
    @staticmethod
    def checkhierarchy(description):
        if "audio_dma" in description["ip"] \
            and "gain_dma" in description["ip"] \
                and "clip_dma" in description["ip"] \
                    and "rect_dma" in description["ip"]:
                        return True
        return False